In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk # Metin verileri üzerinde dil analizi, metin temizleme, kelime frekansı hesaplama, kelime dağarcığı oluşturma gibi NLP işlemlerini gerçekleştirmek için kullanılır.
import re #Metin içinde desenleri aramak, eşleşen desenleri bulmak, metinleri bölümlere ayırmak ve metinleri değiştirmek 
from sklearn.feature_extraction.text import CountVectorizer  # Metin verilerini sayısal özelliklere dönüştürmek için kullanılan bir sınıftır.
from sklearn.ensemble import RandomForestClassifier # birçok karar ağacının bir araya gelerek oluşturduğu bir sınıflandırma veya regresyon modelidir.
from sklearn.metrics import roc_auc_score # Sınıflandırma modelinin performansını ölçmek için kullanılan bir metriktir
from bs4 import BeautifulSoup # HTML ve XML gibi belgeleri analiz etmek ve çıkarımlar yapmak için kullanılan bir kütüphanedir
from sklearn.model_selection import train_test_split # Veri kümesini eğitim ve test alt kümelerine ayırmak için kullanılan bir fonksiyondur
from nltk.corpus import stopwords # Bu kütüphane, dilbilgisi, bağlaçlar, zamirler gibi yaygın kullanılan ancak genellikle metin analizi açısından önemsiz kabul edilen kelimeleri içerir.

In [2]:
df=pd.read_csv("NLPlabeledData.tsv",delimiter="\t",quoting=3)

In [3]:
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


###### IMDB Filim yorumlarını ele alarak olumlu yada olumsuz bir cumle olup olmadığına gösteren bir verimiz var  biz bu verileri doğal dil işleme algoritması ile eğitip test edecez

In [4]:
df.shape # veri setimizin boyutuna bakalım 

(25000, 3)

In [5]:
df.info() # verilerimiz hakında biraz bilgi edinelim 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [6]:
df.isnull().sum() # böş değerlerimizin olup olmadığına bakalım

id           0
sentiment    0
review       0
dtype: int64

In [7]:
colum=["id","duygu_durumunu","yorum"]
df.columns=colum # dahada anlaşılması için değişkenlerin TR yazıyorum 

In [8]:
df.head()

,id,duygu_durumunu,yorum
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [9]:
nltk.download("stopwords")  # Metin analizi açısından önemsiz kabul edilen kelimeleri temizlemek için yüklüyoruz


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abdulbarisoylemez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
örnek_yorum=df.yorum[0] # ilk yorumumuzu alalım ve veriyi işlenecek hale getirelim 
öy=örnek_yorum
öy

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [11]:
öy=BeautifulSoup(öy).get_text() # HTML tagları temzilemek için kulandık
öy

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 2

In [12]:
anlamsız_karakterler=re.sub("[a-zA-Z]"," ",öy) # verimiz de olan anlamsız karekterler 
anlamsız_karakterler 

'"                                                      \'                                 ,                                            ,                                             .                                                                                                                                                                      .                             ,                                                                                              .                                        \'                                                                           \'   .                                                                                                                                                               .                                                                                                                                                                              .                                                                  2

##### şimdi bu anlamsız karekterleri çıkartalım

In [13]:
öy=re.sub("[^a-zA-Z]"," ",öy) # [^a-zA-Z] => a dan z ye ve A dan Z kadar olan kelimler alınsın dışındakiler atılsın
öy

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    m

In [14]:
öy=öy.lower() # makine öğrenmesinde aynı harfin buyük kücük oluşu farklı sonuclar çıkartacaktır bu yuzden bütün hafleri küçük yapıyoruz
öy

' with all this stuff going down at the moment with mj i ve started listening to his music  watching the odd documentary here and there  watched the wiz and watched moonwalker again  maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring  some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for    m

#####  şimdi stopwords kelimeleri ( yani , the , is , an, bibi kelimleri  ) yapay zeka tarafından kulnılmsını istemiyoruz  bunun için kelimleri ayırıyoruz  bunu için split metodu kulanılır 


In [15]:
öy=öy.split()

In [16]:
print(öy)

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again', 'maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent', 'moonwalker', 'is', 'part', 'biography', 'part', 'feature', 'film', 'which', 'i', 'remember', 'going', 'to', 'see', 'at', 'the', 'cinema', 'when', 'it', 'was', 'originally', 'released', 'some', 'of', 'it', 'has', 'subtle', 'messages', 'about', 'mj', 's', 'feeling', 'towards', 'the', 'press', 'and', 'also', 'the', 'obvious', 'message', 'of', 'drugs', 'are', 'bad', 'm', 'kay', 'visually', 'impressive', 'but', 'of', 'course', 'this', 'is', 'all', 'about', 

In [17]:
len(öy) # bunu gösterme sebebim bi sornraki adımda stopword kulanarak azalacak anlamsız(algoritma için) kelimelerdir 

437

In [18]:
sword=set(stopwords.words("english")) # ingiliz dilinde stopword olan kelimeleri sword a set etik ekledik 
öy=[w for w in öy if w not in sword]  # şimdi bunu bizim öy( örenk yorum ) da kaç tanesi ile ayı olmayanı alıyoruz 
öy

['stuff',
 'going',
 'moment',
 'mj',
 'started',
 'listening',
 'music',
 'watching',
 'odd',
 'documentary',
 'watched',
 'wiz',
 'watched',
 'moonwalker',
 'maybe',
 'want',
 'get',
 'certain',
 'insight',
 'guy',
 'thought',
 'really',
 'cool',
 'eighties',
 'maybe',
 'make',
 'mind',
 'whether',
 'guilty',
 'innocent',
 'moonwalker',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'remember',
 'going',
 'see',
 'cinema',
 'originally',
 'released',
 'subtle',
 'messages',
 'mj',
 'feeling',
 'towards',
 'press',
 'also',
 'obvious',
 'message',
 'drugs',
 'bad',
 'kay',
 'visually',
 'impressive',
 'course',
 'michael',
 'jackson',
 'unless',
 'remotely',
 'like',
 'mj',
 'anyway',
 'going',
 'hate',
 'find',
 'boring',
 'may',
 'call',
 'mj',
 'egotist',
 'consenting',
 'making',
 'movie',
 'mj',
 'fans',
 'would',
 'say',
 'made',
 'fans',
 'true',
 'really',
 'nice',
 'actual',
 'feature',
 'film',
 'bit',
 'finally',
 'starts',
 'minutes',
 'excluding',
 'smooth',
 'crim

In [19]:
len(öy) # stopword kulanarak algoritma için anlamsız olan kelimelerin hepsi çıkartıldı bir nevi yarıya kadar düştü 

219

### şimdi bu işlemleri bütün yorumlar için gerçekleştirelim 

In [20]:
def VerTem(yorum):
    yorum=BeautifulSoup(yorum).get_text() #HTML tagları temzilemek için 
    yorum=re.sub("[^a-zA-Z]"," ",yorum) # Anlamsız karekterleri çıkarmak için
    yorum=yorum.lower() # Her harfi küçük yazzlım
    yorum=yorum.split() # stopword için her kelimeyi ayrı yazalım 
    sword=set(stopwords.words("english")) # ingiliz dilinde stopword olan kelimeleri sword a set etik ekledik 
    yorum=[w for w in yorum if w not in sword] # verimizde stopword kelimeleri çıkartalım
    return(" ".join(yorum))

#### şimdi fonkiyonu hayata geçirelim her yorum için

In [21]:
TumYor=[]   # boş bir tane liste oluşturduk

for i in range(len(df["yorum"])):
    if (i+1)%1000==0:
        print((i+1)," tane yorum incelendi..")  # işlem sayısını gösterecek 
    TumYor.append(VerTem(df["yorum"][i])) # temizlenen veriyi yeni listeye ekliyoruz 
    

/Users/abdulbarisoylemez/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


1000  tane yorum incelendi..
2000  tane yorum incelendi..
3000  tane yorum incelendi..
4000  tane yorum incelendi..
5000  tane yorum incelendi..
6000  tane yorum incelendi..
7000  tane yorum incelendi..
8000  tane yorum incelendi..
9000  tane yorum incelendi..
10000  tane yorum incelendi..
11000  tane yorum incelendi..
12000  tane yorum incelendi..
13000  tane yorum incelendi..
14000  tane yorum incelendi..
15000  tane yorum incelendi..
16000  tane yorum incelendi..
17000  tane yorum incelendi..
18000  tane yorum incelendi..
19000  tane yorum incelendi..
20000  tane yorum incelendi..
21000  tane yorum incelendi..
22000  tane yorum incelendi..
23000  tane yorum incelendi..
24000  tane yorum incelendi..
25000  tane yorum incelendi..


###### Şimdi verilerimizi Eğitim ve Test olarak belirliyelim 

In [22]:
x=TumYor    # temizlediğimiz tum yorumlar 
y=np.array(df["duygu_durumunu"])   # yorumun olumsuz mu olumlu mu olduğu y değişkenine atıyoruz 

# train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)


### BAG of WORD matrisi
###### kelimelerin metinlerdeki geçme frekanslarına dayanan ve kelimeleri vektör olarak temsil eden en temel yöntemdir . Bu yöntem, NLP derslerinin her seviyesinde anlatılan ve anlaşılması oldukça kolay olan yaklaşımlardır , bu en temel anlamda yapay zeka metinleri anlamaz bunun yerine onun anlayacağı 1 ve 0 matrisine çevirecez 

In [23]:
countvector=CountVectorizer( max_features=5000) # bag of word boyutu 5000 yaptık 

In [24]:
x_train=countvector.fit_transform(x_train)  # eğitim verilerimizi bag of word matrixine çeviriyoruz 

In [25]:
x_train # yani <20000x5000 sparse matrix of type '<class 'numpy.int64'>' değişkeni bize sutun sayısı 5000 satır sayısı 20000 ve  tipi int olduğunu söyluyor 

<21250x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 1677632 stored elements in Compressed Sparse Row format>

In [26]:
x_train=x_train.toarray() # fit işlemi için array e çeviriyoruz 

In [27]:
ytrain=y_train
x_train.shape,ytrain.shape

((21250, 5000), (21250,))

###  Random forest modeli oluşturalım  ve fit edelim 
###### Random Forest modeli, birçok karar ağacının bir araya gelerek oluşturduğu bir makine öğrenmesi modelidir ve tahminlerin ortalamasını alarak yüksek performanslı ve genelleyici sonuçlar elde etmeyi amaçlar.

In [28]:
model=RandomForestClassifier(n_estimators=100) #  n_estimators karar ağacı sayısını belirtir.
model.fit(x_train,y_train) # modeli eğitiyoruz 


RandomForestClassifier()

### şimdi sıra test datamızda 

In [29]:
xx_test=countvector.transform(x_test) # test verilerimizi bag of word matrixine çeviriyoruz 

In [30]:
xx_test

<3750x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 297527 stored elements in Compressed Sparse Row format>

In [31]:
# test bag of woerd u matrixe çeviriyoruz 
xx_test = xx_test.toarray()

In [32]:
xx_test.shape

(3750, 5000)

#### Predict kısımı
"predict" terimi, bir modelin eğitim sürecinden sonra, yeni ve görülmemiş veriler üzerinde tahmin yapması anlamına gelir. Model, eğitim veri seti üzerinde öğrendiği desenleri kullanarak, girdi olarak verilen yeni veri noktaları için bir çıktı değeri veya sınıflandırma tahmini üretir.

In [33]:
testpredic=model.predict(xx_test)
doğruluk=roc_auc_score(y_test,testpredic)

In [34]:
print("Doğruluk oranı : %",doğruluk)

Doğruluk oranı : % 0.8511806252630782
